# PyCitySchools

In [1]:
import pandas as pd

In [2]:
# Paths to collect the data files.
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [3]:
# Read in CSV files.
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [4]:
# Merge data files.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

In [41]:
# Calculate totals for the whole dataset to get a district summary.
schools_count = school_data["school_name"].count()
students_count = school_data["size"].sum()
total_budget = school_data["budget"].sum()
math_average = student_data["math_score"].mean()
read_average = student_data["reading_score"].mean()
per_pass_math = sum(student_data["math_score"] >= 70) / students_count * 100
per_pass_read = sum(student_data["reading_score"] >= 70) / students_count * 100
per_pass_both = sum((student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70)) / students_count * 100

In [45]:
# Store variables in a dataframe, and display results.
district_summary = pd.DataFrame({"Total Schools": [schools_count],
                                 "Total Students": [students_count],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [math_average],
                                 "Average Reading Score": [read_average],
                                 "% Passing Math": [per_pass_math],
                                 "% Passing Reading": [per_pass_read],
                                 "% Overall Passing": [per_pass_both]})
district_summary.style.format({"Total Students": "{:,.0f}", "Total Budget": "${:,.2f}"}).hide_index()

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [52]:
# Calculate totals for each school, using groupby.
school_type = school_data_complete.groupby("school_name")["type"].unique().str[0]
total_students = school_data_complete.groupby("school_name")["Student ID"].count()
total_budget = school_data_complete.groupby("school_name")["budget"].unique().str[0]
per_student_budget = (total_budget / total_students)
math_average = school_data_complete.groupby("school_name")["math_score"].mean()
read_average = school_data_complete.groupby("school_name")["reading_score"].mean()

pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70, ["school_name","math_score"]]
per_pass_math = pass_math.groupby("school_name")["math_score"].count() / total_students * 100

pass_read = school_data_complete.loc[school_data_complete["reading_score"] >= 70, ["school_name","reading_score"]]
per_pass_read = pass_read.groupby("school_name")["reading_score"].count() / total_students * 100

pass_both = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70), ["school_name","math_score","reading_score"]]
per_pass_both = pass_both.groupby("school_name")["math_score"].count() / total_students * 100

In [65]:
# Store variables in a dataframe, and display results.
school_summary = pd.DataFrame({"School Type": school_type,
                               "Total Students": total_students,
                               "Total School Budget": total_budget,
                               "Per Student Budget": per_student_budget,
                               "Average Math Score": math_average,
                               "Average Reading Score": read_average,
                               "% Passing Math": per_pass_math,
                               "% Passing Reading": per_pass_read,
                               "% Overall Passing": per_pass_both})
school_summary.index.name = ''
school_summary.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

In [72]:
# Sort results in descending order by % Overall Passing, and display top 5.
top_schools = school_summary.sort_values("% Overall Passing", ascending = False).head(5)
top_schools.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [73]:
# Sort results in ascending order by % Overall Passing, and display top 5 (will show lowest scoring).
bottom_schools = school_summary.sort_values("% Overall Passing", ascending = True).head(5)
bottom_schools.style.format({"Total School Budget": "${:,.2f}", "Per Student Budget": "${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [70]:
# Create lists for each grade, keeping school name and math score.
math_grade9 = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name","math_score"]]
math_grade10 = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name","math_score"]]
math_grade11 = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name","math_score"]]
math_grade12 = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name","math_score"]]

# Calculate average math scores by grade.
av_math_grade9 = math_grade9.groupby("school_name")["math_score"].mean()
av_math_grade10 = math_grade10.groupby("school_name")["math_score"].mean()
av_math_grade11 = math_grade11.groupby("school_name")["math_score"].mean()
av_math_grade12 = math_grade12.groupby("school_name")["math_score"].mean()

# Put results in a dataframe and display.
math_scores_by_grade = pd.DataFrame ({"9th": av_math_grade9, 
                                      "10th": av_math_grade10,
                                      "11th": av_math_grade11,
                                      "12th": av_math_grade12})
math_scores_by_grade.index.name = ''
math_scores_by_grade

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade

In [71]:
# Create lists for each grade, keeping school name and reading score.
read_grade9 = school_data_complete.loc[school_data_complete["grade"] == "9th", ["school_name","reading_score"]]
read_grade10 = school_data_complete.loc[school_data_complete["grade"] == "10th", ["school_name","reading_score"]]
read_grade11 = school_data_complete.loc[school_data_complete["grade"] == "11th", ["school_name","reading_score"]]
read_grade12 = school_data_complete.loc[school_data_complete["grade"] == "12th", ["school_name","reading_score"]]

# Calculate average reading scores by grade.
av_read_grade9 = read_grade9.groupby("school_name")["reading_score"].mean()
av_read_grade10 = read_grade10.groupby("school_name")["reading_score"].mean()
av_read_grade11 = read_grade11.groupby("school_name")["reading_score"].mean()
av_read_grade12 = read_grade12.groupby("school_name")["reading_score"].mean()

# Put results in a dataframe and display.
read_scores_by_grade = pd.DataFrame ({"9th": av_read_grade9, 
                                      "10th": av_read_grade10,
                                      "11th": av_read_grade11,
                                      "12th": av_read_grade12})
read_scores_by_grade.index.name = ''
read_scores_by_grade

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [13]:
# Create ranges for school spending per student, and cut data based on bins.
bins = [0, 584, 629, 644, 680]
group_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], bins, labels=group_labels)
# Group data by ranges and calculate averages.
spend_group = school_summary.groupby("Spending Ranges (Per Student)")
av_math_score = spend_group["Average Math Score"].mean()
av_read_score = spend_group["Average Reading Score"].mean()
per_pass_math = spend_group["% Passing Math"].mean()
per_pass_read = spend_group["% Passing Reading"].mean()
per_pass_both = spend_group["% Overall Passing"].mean()

In [14]:
# Store results in a dataframe and display.
scores_by_school_spend = pd.DataFrame({"Average Math Score": av_math_score,
                                       "Average Reading Score": av_read_score,
                                       "% Passing Math": per_pass_math,
                                       "% Passing Reading": per_pass_read,
                                       "% Overall Passing": per_pass_both})
scores_by_school_spend

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [15]:
# Create ranges for school size, and cut data based on bins.
bins = [0, 999, 1999, 5000]
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary["School Size"] = pd.cut(school_summary["Total Students"], bins, labels=group_labels)
# Group data by ranges and calculate averages.
size_group = school_summary.groupby("School Size")
av_math_score = size_group["Average Math Score"].mean()
av_read_score = size_group["Average Reading Score"].mean()
per_pass_math = size_group["% Passing Math"].mean()
per_pass_read = size_group["% Passing Reading"].mean()
per_pass_both = size_group["% Overall Passing"].mean()

In [16]:
# Store results in a dataframe and display.
scores_by_school_size = pd.DataFrame({"Average Math Score": av_math_score,
                                       "Average Reading Score": av_read_score,
                                       "% Passing Math": per_pass_math,
                                       "% Passing Reading": per_pass_read,
                                       "% Overall Passing": per_pass_both})
scores_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [17]:
# Group data by school type and calculate averages.
type_group = school_summary.groupby("School Type")
av_math_score = type_group["Average Math Score"].mean()
av_read_score = type_group["Average Reading Score"].mean()
per_pass_math = type_group["% Passing Math"].mean()
per_pass_read = type_group["% Passing Reading"].mean()
per_pass_both = type_group["% Overall Passing"].mean()

In [18]:
# Store results in a dataframe and display.
scores_by_school_type = pd.DataFrame({"Average Math Score": av_math_score,
                                       "Average Reading Score": av_read_score,
                                       "% Passing Math": per_pass_math,
                                       "% Passing Reading": per_pass_read,
                                       "% Overall Passing": per_pass_both})
scores_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


## Conclusions

A conclusion that can be drawn from the above analysis is that Charter schools perform better than District schools in both Math and Reading. Not only are both the average math and reading scores higher for Charter schools, but the percentage of students who pass is also significantly higher, with Charter schools seeing 90.4% of students pathing both subjects, compared to 53.7% in District schools.

Another conclusion that can be drawn is that schools who spend less per student perform better than those that spend more. The table analysing school scores by spending illustrates that both average scores and pass rate decreases as the spending range increases. This conclusion is further proven as we can see the top performing school, Cabrera High School, spends $582 per student and so would fit into the lowest spending band.

Finally, it can be seen that large schools (greater than 2000 students) perform significantly worse in both average scores and pass rates, than smaller schools.